# 7.10 구현하기: 간단한 합성곱 신경망 클래스

## 7.10.1 기반 클래스 파일 실행시키기

In [2]:
%run ../chap06/adam_model.ipynb

## 7.10.2 CnnBasicModel 클래스 선언

In [3]:
class CnnBasicModel(AdamModel):
    def __init__(self, name, dataset, hconfigs, show_maps = False):
        if isinstanec(hconfigs, list) and \
        not isinstance(hconfigs[0], (list, int)): # hconfigs[0] 값의 유형을 검사하여(첫번째값이 list도 int도 아닌 경우?)
                                                  # 경우에 따라 리스트 껍질을 한 번 더 씌움
            hconfigs = [hconfigs]
        self.show_maps = show_maps # 커널과 특징맵을 시각화 대상으로 포함할지 여부
        self.need_maps = False # 현재 특징맵 수집이 필요한지 여부 (시각화단계)
        self.kernals = [] # 커널 저장 리스트
        
        super(CnnBasicModel, self).__init__(name, dataset, hconfigs)
        self.use_adam = True

## 7.10.3 계층에 대한 파라미터 생성 메서드 재정의

In [4]:
def cnn_basic_alloc_layer_param(self, input_shape, hconfig):
    layer_type = get_layer_type(hconfig)
    
    m_name = 'alloc_{}_layer'.format(layer_type)
    method = getattr(self, m_name)
    pm, output_shape = method(input_shape, hconfig)
    
    return pm, output_shape

CnnBasicModel.alloc_layer_param = cnn_basic_alloc_layer_param

## 7.10.4 계층에 대한 순전파 메서드 재정의

In [5]:
def cnn_basic_forward_layer(self, x, hconfig, pm):
    layer_type = get_layer_type(hconfig)
    
    m_name = 'forward_{}_layer'.format(layer_type)
    method = getattr(self, m_name)
    y, aux = method(x, hconfig, pm)
    
    return y, aux

CnnBasicModel.forward_layer = cnn_basic_forward_layer

## 7.10.5 계층에 대한 역전파 메서드 재정의

In [6]:
def cnn_basic_backprop_layer(self, G_y, hconfig, pm, aux):
    layer_type = get_layer_type(hconfig)
    
    m_name = 'backprop_{}_layer'.format(layer_type)
    method = getattr(self, m_name)
    
    G_input = method(G_y, hconfig, pm, aux)
    
    return G_input

CnnBasicModel.backprop_layer = cnn_basic_backprop_layer 

## 7.10.6 네 가지 계층에 대한 파라미터 생성 메서드 정의

In [7]:
def cnn_basic_alloc_full_layer(self, input_shape, hconfig):
    input_cnt = np.prod(input_shape)
    output_cnt = get_conf_param(hconfig, 'width', hconfig) # hconfig의 두 번째 딕셔너리에서 'width'값 즉, 출력 개수를 반환한다.
                                                           # 이때 세 번째 변수는 다양한 예외 상황에서의 defalut_value이다.
    
    weight = np.random.normal(0, self.rand_std, [intput_cnt, output_cnt])
    bias = np.zeros([output_cnt])
    
    return {'w':weight, 'b':bias}, [output_cnt] # 다차원 출력 형태를 갖는 새로운 계층들과 일관성을 갖도록 리스트에 반환([output_cnt])

def cnn_basic_alloc_conv_layer(self, input_shape, hconfig):
    assert len(input_shape) == 3
    xh, xw, xchn = input_shape
    kh, kw = get_conf_param_2d(hconfig, 'ksize')
    ychn = get_conf_param(hconfig, 'chn')
    
    kernel = np.random.normal(0, self.rand_std, [kh, kw, xchn, ychn])
    bias = np.zeros([ychn])

    if self.show_maps: self.kernels.append(kernel)
        
    return {'k':kernel, 'b':bias}, [xh, xw, ychn]

def cnn_basic_alloc_pool_layer(self, input_shape, hconfig):
    assert len(input_shape) == 3
    xh, xw, xchn = input_shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    
    assert xh % sh == 0
    assert xw % sw == 0
    
    return {}, [xh//sh, xw//sw, xchn]

CnnBasicModel.alloc_full_layer = cnn_basic_alloc_full_layer
CnnBasicModel.alloc_conv_layer = cnn_basic_alloc_conv_layer
CnnBasicModel.alloc_max_layer = cnn_basic_alloc_pool_layer
CnnBasicModel.alloc_avg_layer = cnn_basic_alloc_pool_layer

## 7.10.7 은닉 계층 구성 정보 접근을 위한 보조 함수 정의

In [8]:
def get_layer_type(hconfig):
    if not isinstance(hconfig, list): return 'full'
    return hconfig[0]

def get_conf_param(hconfig, key, defval = None):
    if not isinstance(hconfig, list): return defval
    if len(hconfig) <= 1: return defval
    if not key in hconfig[1]: return defval
    return hconfig[1][key]

def get_conf_param_2d(hconfig, key, defval = None):
    if len(hconfig) <= 1: return defval
    if not key in hconfig[1]: return defval
    if isinstance(val, list): return val
    return [val, val]

## 7.10.8 완전 연결 계층에 대한 순전파 메서드 정의

In [9]:
def cnn_basic_forward_full_layer(self, x, hconfig, pm):
    if pm is None: return x, None # GAN에서 출력 계층의 동작을 무력화해야 하는 경우를 지원하기 위한 것
    
    x_org_shape = x.shape # # 역전파에서 원래의 입력 형태가 필요하므로
    
    if len(x.shape) != 2: # 입력이 2차원 형태가 아닌 경우,
        mb_size = x.shape[0] 
        x = x.reshape([mb_size, -1]) # 미니배치 크기 x 나머지로 차원 축소시킴
        
    affine = np.matmul(x, pm['w']) + pm['b']
    y = self.activate(affine, hconfig)
    
    return y, [x, y, x_org_shape]

CnnBasicModel.forward_full_layer = cnn_basic_forward_full_layer

## 7.10.9 완전 연결 계층에 대한 역전파 메서드 정의

In [10]:
def cnn_basic_backprop_full_layer(self, G_y, hconfig, pm, aux):
    if pm is None: return G_y # GAN 모델을 처리할 때 출력 계층의 동작을 무력화하기 위함
    
    x, y, x_org_shape = aux
    
    G_affine = self.activate_derv(G_y, y, hconfig) # 비선형 활성화 함수의 역전파 처리를 수행하는 함수 호출
    
    g_affine_weight = x.transpose()
    g_affine_input = pm['w'].transpose()
    
    G_weight = np.matmul(g_affine_weight, G_affine)
    G_bias = np.sum(G_affine, axis = 0)
    G_input = np.matmul(G_affine, g_affine_input)
    
    self.update_param(pm, 'w', G_weight)
    self.update_param(pm, 'b', G_bias)
    
    return G_input.reshape(x_org_shape)

CnnBasicModel.backprop_full_layer = cnn_basic_backprop_full_layer

## 7.10.10 비선형 활성화 함수에 대한 순전파 및 역전파 메서드 정의

In [11]:
def cnn_basic_activate(self, affine, hconfig):
    if hconfig is None: return affine
    
    func = get_conf_param(hconfig, 'actfunc', 'relu')
    
    if func == 'none':      return affine
    elif func == 'relu':    return relu(affine)
    elif func == 'sigmoid': return sigmoid(affine)
    elif func == 'tanh':    return tanh(affine)
    else:                   assert 0
        
def cnn_basic_activate_derv(self, G_y, y, hconfig):
    if hconfig is None: return G_y
    
    func = get_conf_param(hconfig, 'actfunc', 'relu')
    
    if func == 'none':      return G_y
    elif func == 'relu':    return relu_derv(y) * G_y
    elif func == 'sigmoid': return sigmoid_derv(y) * G_y
    elif func == 'tanh':    return tanh_derv(y) * G_y
    else:                   assert 0 
        
CnnBasicModel.activate = cnn_basic_activate
CnnBasicModel.activate_derv = cnn_basic_activate_derv

## 7.10.11 합성곱 계층에 대한 원론적 순전파 처리 함수 정의

In [12]:
def forward_conv_layer_adhoc(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    conv = np.zeros((mb_size, xh, xw, ychn))
    
    for n in range(mb_size):
        for r in range(xh):
            for c in range(xw):
                for ym in range(ychn):
                    for i in range(kh): # 대응하는 입력 픽셀값과 커널 가중치를 찾아 그 곱을 누적시킴
                        for j in range(kw):
                            rx = r + i - (kh-1) // 2 
                            cx = c + j - (kw-1) // 2
                            if rx < 0 or rx >= xh: continue
                            if cx < 0 or cx >= xw: continue
                            for xm in range(xchn):
                                kval = pm['k'][i][j][xm][ym]
                                ival = x[n][rx][cx][xm]
                                conv[n][r][c][ym] += kval * ival
    y = self.activate(conv + pm['b'], hconfig) # 4차원 텐서에 편향 벡터를 일괄적으로 더하는 방법
    
    return y, [x, y]

## 7.10.12 합성곱 계층에 대한 개선된 순전파 처리 함수 정의

In [13]:
def forward_conv_layer_better(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    conv = np.zeros((mb_size, xh, xw, ychn))
    
    bh, bw = (kh-1)//2, (kw-1)//2
    eh, ew = xh + kh - 1, xw+ kw - 1
    
    x_ext = np.zeros((mb_size, eh, ew, xchn)) # 커널 크기를 고려해 확장된 x_ext 버퍼를 준비
    x_ext[:, bh:bh + xh, bw:bw + xw, :] = x # 중앙 부분에 입력(x) 값을 복사
    
    k_flat = pm['k'].transpose([3, 0, 1, 2]).reshape([ychn, -1]) # 출력 채널 별로 커널의 나머지 세 차원을 한 차원 벡터로 차원 축소
    
    for n in range(mb_size):
        for r in range(xh):
            for c in range(xw):
                for ym in range(ychn):
                    xe_flat = x_ext[n, r:r + kh, c:c + kw, :].flatten()
                    conv[n, r, c, ym] = (xe_flat * k_flat[ym]).sum()
    y = self.activate(conv + pm['b'], hconfig)
    
    return y, [x, y]

## 7.10.13 합성곱 계층에 대한 순전파 메서드 정의

In [14]:
def cnn_basic_forward_conv_layer(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    x_flat = get_ext_regions_for_conv(x, kh, kw)
    k_flat = pm['k'].reshape([kh*kw*xchn, ychn])
    conv_flat = np.matmul(x_flat, k_flat)
    conv = conv_flat.reshape([mb_size, xh, xw, ychn])
    
    y = self.activate(conv + pm['b'], hconfig)
    
    if self.need_maps: self.maps.append(y)
        
    return y, [x_flat, k_flat, x, y]

CnnBasicModel.forward_conv_layer = cnn_basic_forward_conv_layer

## 7.10.14 합성곱 계층에 대한 역전파 메서드 정의

In [15]:
def cnn_basic_backprop_conv_layer(self, G_y, hconfig, pm, aux):
    x_flat, k_flat, x, y = aux
    
    kh, kw, xchn, ychn = pm['k'].shape
    mb_size, xh, xw, _ = G_y.shape
    
    G_conv = self.activate_derv(G_y, y, hconfig)
    
    G_conv_flat = G_conv.reshape(mb_size*xh,xw, ychn)
    
    g_conv_k_flat = x_flat.transpose()
    g_conv_x_flat = k_flat.transpose()
    
    G_k_flat = np.matmul(g_conv_k_flat, G_conv_flat)
    G_x_flat = np.matmul(G_conv_flat, g_conv_x_flat)
    G_bias = np.sum(G_conv_flat, axis = 0)
    
    G_kernel = G_k_flat.reshape([kh, kw, xchn, ychn])
    G_input = undo_ext_regions_for_conv(G_x_flat, x, kh, kw)
    
    self.update_param(pm, 'k', G_kernel)
    self.update_param(pm, 'b', G_bias)
    
    return G_input

CnnBasicModel.backprop_conv_layer = cnn_basic_backprop_conv_layer

## 7.10.15 확장 영역 처리 함수 정의

In [16]:
def get_ext_regions_for_conv(x, kh, kw):
    mb_size, xh, xw, xchn = x.shape
    
    regs = get_ext_regions(x, kh, kw, 0)
    regs = regs.transpose([2, 0, 1, 3, 4, 5]) # 데이터 순서를 바꿈
    
    return regs.reshape([mb_size*xh*xw, kh*kw*xchn]) # 데이터 형태도 바꿔서 반환

def get_ext_regions(x, kh, kw, fill): # 두 함수를 따로 분리하여 처리하는 이유: 나중에 확장된 풀링 연산을 지원할 때에도 이 기능이 필요함
    mb_size, xh, xw, xchn = x.shape
    
    eh, ew = xh + kh - 1, xw + kw - 1
    bh, bw = (kh-1)//2 ,(kw-1)//2
    
    x_ext = np.zeros((mb_size, eh, ew, xchn), dtype = 'float32') + fill # 확장된 풀링 연산에서는 0이 아닌 다른 값을 확장 영역에 채워야 함
    x_ext[:, bh:bh + xh, bw:bw+ xw, :] = x
    
    regs = np.zeros((xh, xw, mb_size*kh*kw*xchn), dtype = 'float32') # 3차원 텐서를 준비
    
    for r in range(xh):
        for c in rnage(xw):
            regs[r, c, :] = x_ext[:, r:r + kh, c:c + kw, :].flatten() # 이미지의 픽셀 좌표별로 커널 크기의 인근 영역에 대해 미니배치
                                        # 데이터 전체와 입력 채널 전체 등 4차원 공간에서 해당되는 원소를 찾아 일괄적으로 차원 축소시킴
            
    return regs.reshape([xh, xw, mb_size, kh, kw, xchn]) # 우선 데이터 순서를 유지한 채 텐서를 반환

## 7.10.16 확장 영역에 대한 역처리 함수 정의

In [17]:
def undo_ext_regions_for_conv(regs, x, kh, kw):
    mb_size, xh, xw, xchhn = x.shape
    
    regs = regs.reshape([mb_size, xh, xw, kh, kw, xchn])
    regs = regs.transpose([1, 2, 0, 3, 4, 5]) # mb_size를 다시 세 번째 차원에 위치시킴
    
    return undo_ext_regions(regs, kh, kw)

def undo_ext_regions(regs, kh, kw):
    xh, xw, mb_size, kh, kw, xchn = regs.shape
    
    eh, ew = xh + kh - 1, xw + kw - 1
    bh, bw = (kh-1)//2, (kw-1)//2
    
    gx_ext = np.zeros([mb_size, eh, ew, xchn], dtype = 'float32') # x_ext의 각 원소의 기울기 텐서
    
    for r in range(xh):
        for c in range(xw):
            gx_ext[:, r:r + kh, c:c + kw, :] += regs[r, c] # 순전파 때 여기저기 중복하여 이용되는 성분의 손실기울기를 더함
    
    return gx_ext[:, bh:bh + xh, bw:bw + xw, :]

## 7.10.17 평균치 풀링 계층에 대한 순전파와 역전파 메서드 정의

In [18]:
def cnn_basic_forward_avg_layer(self, x, hconfig, pm):
    mb_size, xh, xw, chn = x.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    yh, yw = xh // sh, xw // sw
    
    x1 = x.reshape([mb_size, yh, sh, yw, sw, chn])
    x2 = x1.transpose(0, 1, 3, 5, 2, 4) # 보폭을 나타내는 sh, sw를 뒤로 뺌
    x3 = x2.reshape([-1, sh*sw]) # 2차원으로 차원을 축소시킴
    
    y_flat = np.average(x3, 1)
    y = y_flat.reshape([mb_size, yh, yw, chn])
    
    if self.need_maps: self.maps.append(y) 
    
    return y, None # None?

def cnn_basic_backprop_avg_layer(self, G_y, hconfig, pm, aux):
    mb_size, yh, yw, chn = G_y.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    xh, xw = yh * sh, yw * sw
    
    gy_flat = G_y.flatten() / (sh * sw) # 평균 연산이므로 보폭의 곱으로 미리 나눔
    
    gx1 = np.zeros([mb_size*yh*yw*chn, sh*sw], dtype = 'float32') # 2차원 버퍼 생성
    for i in range(sh*sw):
        gx1[:, i] = gy_flat # 2차원 버퍼의 모든 행에 gy_flat (1 / (sh*sw))을 복사함
    gx2 = gx1.reshape([mb_size, yh, yw, chn, sh, sw])
    gx3 = gx2.transpose([0, 1, 4, 2, 5, 3])
    
    G_input = gx3.reshape([mb_size, xh. xw, chn])
    
    return G_input

CnnBasicModel.forward_avg_layer = cnn_basic_forward_avg_layer
CnnBasicModel.backprop_avg_layer = cnn_basic_back_prop_avg_layer

NameError: name 'cnn_basic_back_prop_avg_layer' is not defined

## 7.10.18 최대치 풀링 계층에 대한 순전파와 역전파 메서드 정의

In [19]:
def cnn_basic_forward_max_layer(self, x, hconfig, pm):
    mb_size, xh, xw, chn = x.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    yh, yw = xh // sh, xw // sw
    
    x1 = x.reshape([mb_size, yh, sh, yw, sw, chn])
    x2 = x1.transpose(0, 1, 3, 5, 2, 4)
    x3 = x2.reshape([-1, sh*sw])
    
    idxs = np.argmax(x3, axis = 1)
    y_flat = x3[np.arange(mb_size*yh*yw*chn), idxs]
    y = y_flat.reshape([mb_size, yh, yw, chn])
    
    if self.need_maps: self.maps.append(y)
        
    return y, idxs #  idxs:역전파에서 사용할 최댓값 위치

def cnn_basic_backprop_max_layer(self, G_y, hconfig, pm, aux):
    idxs = aux
    
    mb_size, yh, yw, chn = G_y.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    xh, xw = yh * sh, yw * sw
    
    gy_flat = G_y.flatten()
    
    gx1 = np.zeros([mb_size*yh*yw*chn, sh*sw], dtype = 'float32')
    gx1[np.arange(mb_size*yh*yw*chn), idxs] = gy_flat # gy_flat[:]에서 [:]을 빼면 안되는지 먼저 해보자
    gx2 = gx1.reshape([mb_size, yh, yw, chn, sh, sw])
    gx3 = gx2.transpose([0, 1, 4, 2, 5, 3])
    
    G_input = gx3.reshape([mb_size, xh, xw, chn])
    
    return G_input

CnnBasicModel.forward_max_layer = cnn_basic_forward_max_layer
CnnBasicModel.backprop_max_layer = cnn_basic_backprop_max_layer

## 7.10.19 시각화 메서드 정의

In [21]:
def cnn_basic_visualize(self, num):
    print('Model {} Visualization'.format(self.name))
    
    self.need_maps = self.show_maps # show_maps(커널과 특징맵이 시각화 대상인지 여부)에 따라 need_maps 변수를 조절
    self.maps = [] # mpas를 새로 초기화
    
    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)
    
    if self.show_maps:
        for kernel in self.kernels:
            kh, kw, xchn, ychn = kernel.shape
            grids = kernel.reshape([kh, kw, -1]).transpose(2, 0, 1)
            draw_images_horz(grids[0:5, :, :])
            
        for pmap in self.maps:
            draw_images_horz(pmap[:, :, :, 0])
            
    self.dataset.visualize(deX, deY, deY)
    
    self.need_maps = False # need_maps를 False로 초기화
    self.maps = None # maps를 None으로 초기화
    
CnnBasicModel.visualize = cnn_basic_visualize